In [4]:
!pip install chromadb

In [5]:
import kagglehub
import pandas as pd
import os
import chromadb
from chromadb.utils import embedding_functions
import time
import multiprocessing as mp

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Download latest version linkedin dataset
# csv format
li = kagglehub.dataset_download("asaniczka/1-3m-linkedin-jobs-and-skills-2024")
dfs = []
for path in os.listdir(li):
    li_p = pd.read_csv(li + '/' + path)
    dfs.append(li_p)
    del li_p

columns = ['job_title','company','job_location','job_level','job_type','job_skills','job_summary']
result = dfs[2].merge(dfs[0], on='job_link').merge(dfs[1], on='job_link')[columns]
print(result.head())

del dfs

import gc ; gc.collect()

# Download latest version resume dataset
# csv format
# rs = kagglehub.dataset_download("snehaanbhawal/resume-dataset")
# rs_p = pd.read_csv(rs + '/Resume/' + os.listdir(rs + '/Resume')[0])
# print(rs_p.head())

def producer(queue, chunk_size=10000):
    for i in range(0, len(result), chunk_size):
        chunk = result[i:i + chunk_size]
        queue.put((chunk['job_summary'].tolist(), [{k: chunk.loc[j, k] for k in columns[:-1]} for j in chunk.index], [str(j) for j in chunk.index]))

100%|██████████| 1.88G/1.88G [01:32<00:00, 21.8MB/s]

Extracting files...


                        job_title                       company  \
0              Restaurant Manager             Rock N Roll Sushi   
1  Med-Surg Registered Nurse (RN)   Touchette Regional Hospital   
2       Registered Nurse Cath Lab  StoneSprings Hospital Center   
3    Commercial Account Executive               The Recruit Lab   
4                   Store Manager                   Stop & Shop   

                               job_location   job_level job_type  \
0                                 Macon, GA  Mid senior   Onsite   
1                           Centreville, IL  Mid senior   Onsite   
2                                Dulles, VA  Mid senior   Onsite   
3  South Yorkshire, England, United Kingdom  Mid senior   Onsite   
4                             Newington, CT  Mid senior   Onsite   

                                          job_skills  \
0  Restaurant Management, Food and Beverage Knowl...   
1  Registered Nurse, Patient Care, Nursing, Healt...   
2  Registered Nurse,

In [8]:
def consumer(use_cuda, queue):
  # Instantiate chromadb instance. Data is stored on disk (a folder named 'kaggledb' will be created in the same folder as this file).
  chroma_client = chromadb.PersistentClient(path="kaggle_db")
  device = "cuda" if use_cuda else "cpu"

  # Select the embedding model to use
  # List of model names can be found here https://www.sbert.net/docs/pretrained_models.html
  sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2", device=device)

  # Create the collection, aka vector database. Or, if database already exist, then use it. Specify the mmodel that we want to use to do the embedding.
  collection = chroma_client.get_collection(name="linkedin", embedding_function=sentence_transformer_ef)

  while True:
    # Check for items in queue, this process blocks until queue has items to process.
    batch = queue.get()
    if batch is None:
      break

    # Add to collection
    collection.add(
        documents=batch[0],
        metadatas=batch[1],
        ids=batch[2]
    )

In [9]:
if __name__ == '__main__':
  chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/Colab Notebooks")
  sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

  try:
    chroma_client.get_collection(name="linkedin")
    chroma_client.delete_collection(name="linkedin")
  except Exception as err:
    print(err),

  collection = chroma_client.create_collection(name="linkedin", embedding_function=sentence_transformer_ef)

  queue = mp.Queue()

  producer_process = mp.Process(target=producer, args=(queue,))
  consumer_process = mp.Process(target=consumer, args=(True, queue))

  start_time = time.time()

  producer_process.start()
  consumer_process.start()

  producer_process.join()
  queue.put(None)
  consumer_process.join()

  end_time = time.time()

  print(f"Time taken: {end_time - start_time}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-8c7cc6be95ea>", line 11, in consumer
    collection = chroma_client.get_collection(name="linkedin", embedding_function=sentence_transformer_ef)
  File "/usr/local/lib/python3.10/dist-packages/chromadb/api/client.py", line 171, in get_collection
    model = self._server.get_collection(
  File "/usr/local/lib/python3.10/dist-packages/chromadb/telemetry/opentelemetry/__init__.py", line 150, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/chromadb/api/segment.py", line 103, in wrapper
    return self._rate_limit_enforcer.rate_limit(func)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/chromadb/rate_limit/simple_rate_limit/__init__.py", line 23

Time taken: 57.44627594947815
